# Ollama 的使用

## 加载自定义通义千问模型

### curl

In [13]:
%%bash

curl -s http://192.168.0.72:11434/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "xiaoyu",
        "messages": [
            {
                "role": "user",
                "content": "你是谁"
            }
        ]
    }' | jq .

{
  "id": "chatcmpl-642",
  "object": "chat.completion",
  "created": 1714988722,
  "model": "xiaoyu",
  "system_fingerprint": "fp_ollama",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "我是小羽，一个由羽胜科技开发的大型语言模型。我的设计目的是提供信息、解答问题和进行对话，帮助用户在各种主题上获取知识或解决问题。如果您有任何问题或需要帮助，请随时告诉我。"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 0,
    "completion_tokens": 49,
    "total_tokens": 49
  }
}


### openai api

In [2]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://192.168.0.72:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="xiaoyu",
  messages=[
    {"role": "system", "content": "You are a helpful assistant. 回答要用中文"},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

2020年的世界大赛在洛杉矶道奇队的主场，Dodger Stadium进行。由于COVID-19的大流行，常规的赛季和季后赛模式有所调整，常规的球迷限制和健康安全协议实行，导致比赛是在空无观众的情况下进行的。


## 加载bge-m3嵌入模型

### curl

In [8]:
%%bash

curl -s http://192.168.0.72:11435/api/embeddings -d '{
  "model": "bge-m3",
  "prompt": "Llamas are members of the camelid family"
}'

{"embedding":[0.17048664391040802,0.35879766941070557,-1.185837745666504,-0.6105074286460876,0.6739987730979919,0.9419403076171875,-1.1492893695831299,-0.6883870363235474,0.5334706902503967,0.5691338777542114,0.11260024458169937,-0.5420926213264465,0.6517525911331177,-0.24342328310012817,-0.016301140189170837,-0.03544166311621666,0.32418185472488403,0.8689248561859131,0.34042632579803467,-0.16555935144424438,0.4198416769504547,0.018318425863981247,-0.17167139053344727,-0.8792904615402222,-0.4036100208759308,0.5398334860801697,-1.5299004316329956,0.79435795545578,0.4254131019115448,-0.5498369336128235,-0.20647725462913513,0.7418112754821777,-0.47860047221183777,0.43827515840530396,-0.5262548327445984,0.1226063221693039,-0.30673930048942566,-0.08407178521156311,-0.005897290073335171,-0.7875850796699524,0.33499905467033386,0.16250844299793243,0.6812043786048889,-0.2784593105316162,-0.6788148880004883,0.2833603620529175,-0.6161272525787354,0.30833253264427185,-0.31899210810661316,0.7430956

### openai api

In [ ]:
# todo: 目前还不支持，见 https://ollama.com/blog/openai-compatibility More to come

### LlamaIndex

In [6]:
%%time
%%capture

!pip install llama-index
!pip install llama-index-embeddings-ollama

CPU times: user 21.3 ms, sys: 929 µs, total: 22.2 ms
Wall time: 3.78 s


In [4]:
%%time

from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="bge-m3",
    base_url="http://192.168.0.72:11435",
    # ollama_additional_kwargs={"mirostat": 0},
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], 
    # show_progress=True
)
print(pass_embedding)

query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
# print(query_embedding)
len(query_embedding)

[[0.8001663088798523, 0.2519471049308777, -1.6904008388519287, 0.2843567728996277, 0.2337857186794281, 0.8599135279655457, 1.092034935951233, 0.305543452501297, 0.6489359736442566, 0.08954282104969025, 0.3365323841571808, 0.03653690963983536, 0.01787271350622177, 0.25719866156578064, 0.1798175424337387, -0.5444640517234802, 1.0643724203109741, -0.06845273822546005, -0.028119534254074097, 0.3470466136932373, -0.21727700531482697, -0.17143668234348297, -0.08008697628974915, 0.8462682366371155, -0.7251355051994324, 0.24352841079235077, -0.9633706212043762, -0.30624920129776, 0.17286111414432526, -0.07076796889305115, -0.35074421763420105, -0.18377366662025452, -0.5968353152275085, -0.27102720737457275, -0.6890687942504883, -0.5235034227371216, -0.1705496609210968, -1.1042625904083252, -1.0928531885147095, 0.9311663508415222, 0.2652702331542969, -1.083030104637146, -0.36641544103622437, -0.9763517379760742, 0.055924102663993835, -0.058028820902109146, 0.17713677883148193, -0.20752631127834

1024